# Convolutional Neural Network

假设我们有一个足够充分的照片数据集，数据集中是拥有标注的照片，每 张照片具有百万级像素，这意味着网络的每次输入都有一百万个维度。即使将隐藏层维度降低到1000，这个 全连接层也将有106 × 103 = 109个参数。想要训练这个模型将不可实现，因为需要有大量的GPU、分布式优 化训练的经验和超乎常人的耐心。

有些读者可能会反对这个观点，认为要求百万像素的分辨率可能不是必要的。然而，即使分辨率减小为十 万像素，使用1000个隐藏单元的隐藏层也可能不足以学习到良好的图像特征，在真实的系统中我们仍然需要 数十亿个参数。此外，拟合如此多的参数还需要收集大量的数据。然而，如今人类和机器都能很好地区分猫 和狗:这是因为图像中本就拥有丰富的结构，而这些结构可以被人类和机器学习模型使用。

卷积神经网络(convolutional neural networks，CNN)是机器学习利用自然图像中一些已知结构的创造性方法。

1. 首先，多层感知机的输入是二维图像X，其隐藏表示H在数学上是一个矩阵，在代码中表示为二维张量。其中X和H具有相同的形状。为了方便理解，我们可以认为，无论是输入还是隐藏表示都拥有空间结构。

   使用$[X]_{i,j}$ 和$[H]_{i,j}$ 分别表示输入图像和隐藏表示中位置$(i,j)$处的像素。为了使每个隐藏神经元都能接收到 每个输入像素的信息，我们将参数从权重矩阵(如同我们先前在多层感知机中所做的那样)替换为四阶权重 张量$W$。假设$U$包含偏置参数，我们可以将全连接层形式化地表示为
   
    $$\begin{aligned} H_{i,j} =& U_{i,j} + \sum_k \sum_l W_{i,j,k,l} X_{k,l}\\
    =& U_{i,j} + \sum_a \sum_b V_{i,j,a,b} X_{i+a,j+b}\end{aligned}$$



   其中，从W到V的转换只是形式上的转换，因为在这两个四阶张量的元素之间存在一一对应的关系。我们只需 重新索引下标$(k, l)$，使$k = i + a、l = j + b$，由此可得$ [V]_{i,j,a,b} = [W]_{i,j,i+a,j+b} $。索引a和b通过在正偏移和负偏 移之间移动覆盖了整个图像。对于隐藏表示中任意给定位置$(i,j)$处的像素值$H_{i,j}$ ，可以通过在x中以(i, j)为 中心对像素进行加权求和得到，加权使用的权重为$V_{i,j,a,b}$ 。
   
CNN 应该具有以下性质：
1. 平移不变性


现在引用上述的第一个原则:平移不变性。这意味着检测对象在输入X中的平移，应该仅导致隐藏表示H中的 平移。也就是说，$V$和$U$实际上不依赖于$(i, j)$的值，即$[V]_{i,j,a,b} = [V]_{a,b}$。并且$U$是一个常数，比如$u$。因此，我 们可以简化H定义为:

$$Hi,j = u + \sum_a \sum_b V_{a,b} X_{i+a,j+b}$$
这就是卷积(convolution)。我们是在使用系数$V_{a,b}$ 对位置$(i, j )$附近的像素$(i + a, j + b)$进行加权得到$H_{i,j}$ 。
注意，$V_{a,b}$ 的系数比$V_{i,j,a,b}$ 少很多，因为前者不再依赖于图像中的位置。这就是显著的进步!

2. 局部性

为了收集用来训练参数$H_{i,j}$ 的相关信息，我们不应偏离 到距$(i, j)$很远的地方。这意味着在$|a| > ∆$或$|b| > ∆$的范围之外，我们可以设置$V_{a,b} = 0$。因此，我们可以 将$H_{i,j}$ 重写为

$$H_{i,j}=u+\sum^\Delta_{a=-\Delta}\sum^\Delta_{b=-\Delta}V_{a,b}X_{i+a,j+b}$$

这个式子表示的layer就是一个convolutional layer。

$V$被称为卷积核(convolution kernel)或者滤波器(filter)，亦或简单地称之 为该卷积层的权重，通常该权重是可学习的参数。


## convolution

在数学中，两个函数(比如$f,g :\mathbb R_d → \mathbb R$)之间的“卷积”被定义为

$$ (f*g)(\pmb x) = \int f(\pmb z)g(\pmb{x-z})d\pmb z$$

离散情况下与卷积核和表达方式一样。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
 def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)) 
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])

In [4]:
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [5]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [6]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [7]:
K = torch.tensor([[1.0, -1.0]])

In [8]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [9]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 学习卷积核

In [10]:
# 构造一个二维卷积层，它具有1个输出通道和形状为(1，2)的卷积核
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

In [11]:
# 这个二维卷积层使用四维输入和输出格式(批量大小、通道、高度、宽度)， 
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2 # 学习率

In [12]:
for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 7.012
epoch 4, loss 1.981
epoch 6, loss 0.662
epoch 8, loss 0.246
epoch 10, loss 0.097


In [13]:
list(conv2d.parameters())

[Parameter containing:
 tensor([[[[ 0.9584, -1.0216]]]], requires_grad=True)]

## 填充和步幅

- 填充
![title](attachment/cnn_fill.png)

通常，如果我们添加ph 行填充(大约一半在顶部，一半在底部)和pw 列填充(左侧大约一半，右侧一半)，则输出形状将为

$$(n_h −k_h +p_h +1)×(n_w −k_w +p_w +1)$$

在许多情况下，我们需要设置$p_h = k_h − 1$和$p_w = k_w − 1$，使输入和输出具有相同的高度和宽度。

- 步幅
![title](attachment/cnn_stride.png)

通常，当垂直步幅为$s_h$ 、水平步幅为$s_w$ 时，输出形状为

$$⌊(n_h −k_h +p_h +s_h)/s_h⌋×⌊(n_w −k_w +p_w +s_w)/s_w⌋.$$


In [14]:
import torch
from torch import nn
# 为了方便起⻅，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数 
def comp_conv2d(conv2d, X):
    # 这里的(1，1)表示批量大小和通道数都是1 
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略前两个维度:批量大小和通道
    return Y.reshape(Y.shape[2:])

In [15]:
# padding 的取值是行列的两边都添加，2x2的矩阵就会变成4x4
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))

In [16]:
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [17]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [18]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

## channel

当输入包含多个通道时，需要构造一个与输入数据具有相同输入通道数的卷积核，以便与输入数据进行互相 关运算。假设输入的通道数为$c_i$，那么卷积核的输入通道数也需要为$c_i$。如果卷积核的窗口形状是$k_h ×k_w$，那 么当$c_i = 1$时，我们可以把卷积核看作形状为$k_h × k_w$的二维张量。

![title](attachment/cnn_channel.png)

In [19]:
def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度(通道维度)，再把它们加在一起 
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [20]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [21]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [22]:
K.shape

torch.Size([2, 2, 2])

In [23]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [24]:
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

- 1x1 卷积

In [25]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h*w))
    K = K.reshape((c_o, c_i))
    Y = K@X
    return Y.reshape((c_o, h ,w))

In [26]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

In [27]:
Y1 = corr2d_multi_in_out_1x1(X, K)
Y1

tensor([[[ 2.5076, -2.2076,  1.1770],
         [-1.3156,  3.0593,  0.2826],
         [-0.8144, -3.1651,  1.1679]],

        [[ 1.0557, -0.5774,  0.1752],
         [-0.3695, -0.7572, -0.7477],
         [ 0.3195,  0.4934, -0.5823]]])

In [28]:
Y2 = corr2d_multi_in_out(X, K)
Y2

tensor([[[ 2.5076, -2.2076,  1.1770],
         [-1.3156,  3.0593,  0.2826],
         [-0.8144, -3.1651,  1.1679]],

        [[ 1.0557, -0.5774,  0.1752],
         [-0.3695, -0.7572, -0.7477],
         [ 0.3195,  0.4934, -0.5823]]])